# Extract, Clean, and Load Process (MS SQL Server)

In [553]:
from sqlalchemy import create_engine
import zipfile
import os
import numpy as np
import pandas as pd

## Extract data from CSV files

In [554]:
# Path to the zip file
zip_path = "D:/DA_Project/Data/Football_2/archive.zip"
extract_path = "D:/DA_Project/Data/Football_2"

# Extract the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the files after extraction
os.listdir(extract_path)

['appearances.csv',
 'archive.zip',
 'clubs.csv',
 'club_games.csv',
 'competitions.csv',
 'games.csv',
 'game_events.csv',
 'game_lineups.csv',
 'players.csv',
 'player_valuations.csv',
 'transfers.csv']

In [555]:
# Path to the inventory file
appearances_path = os.path.join(extract_path, "appearances.csv")
clubs_path = os.path.join(extract_path, "clubs.csv")
club_games_path = os.path.join(extract_path, "club_games.csv")
competitions_path = os.path.join(extract_path, "competitions.csv")
games_path = os.path.join(extract_path, "games.csv")
game_events_path = os.path.join(extract_path, "game_events.csv")
game_lineups_path = os.path.join(extract_path, "game_lineups.csv")
players_path = os.path.join(extract_path, "players.csv")
player_valuations_path = os.path.join(extract_path, "player_valuations.csv")
transfers_path = os.path.join(extract_path, "transfers.csv")


# Read the data
appearances_df = pd.read_csv(appearances_path, encoding='utf-8')
clubs_df = pd.read_csv(clubs_path, encoding='utf-8')
club_games_df = pd.read_csv(club_games_path, encoding='utf-8')
competitions_df = pd.read_csv(competitions_path, encoding='utf-8')
games_df = pd.read_csv(games_path, encoding='utf-8')
game_events_df = pd.read_csv(game_events_path, encoding='utf-8')
game_lineups_df = pd.read_csv(game_lineups_path, encoding='utf-8')
players_df = pd.read_csv(players_path, encoding='utf-8')
player_valuations_df = pd.read_csv(player_valuations_path, encoding='utf-8')
transfers_df = pd.read_csv(transfers_path, encoding='utf-8')

## Data Cleaning & Preprocessing

### games_df

In [556]:
# Create a copy of the DataFrame for data cleaning
games_df_cleaned = games_df.copy()

In [557]:
# Quickly preview the table data
games_df_cleaned.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321027,L1,2013,1. Matchday,2013-08-11,33.0,41.0,3.0,3.0,8.0,...,Veltins-Arena,61973.0,Manuel Gräfe,https://www.transfermarkt.co.uk/fc-schalke-04_...,4-2-3-1,4-2-3-1,FC Schalke 04,Hamburger SV,3:3,domestic_league
1,2321033,L1,2013,1. Matchday,2013-08-10,23.0,86.0,0.0,1.0,13.0,...,EINTRACHT-Stadion,23000.0,Deniz Aytekin,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-3-1-2,Eintracht Braunschweig,Sportverein Werder Bremen von 1899,0:1,domestic_league
2,2321044,L1,2013,2. Matchday,2013-08-18,16.0,23.0,2.0,1.0,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
3,2321060,L1,2013,3. Matchday,2013-08-25,23.0,24.0,0.0,2.0,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
4,2321072,L1,2013,5. Matchday,2013-09-14,16.0,41.0,6.0,2.0,1.0,...,SIGNAL IDUNA PARK,80645.0,Tobias Welz,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,3-5-2,Borussia Dortmund,Hamburger SV,6:2,domestic_league


In [558]:
# All columns in games_df
games_df_cleaned.columns

Index(['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id',
       'away_club_id', 'home_club_goals', 'away_club_goals',
       'home_club_position', 'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url',
       'home_club_formation', 'away_club_formation', 'home_club_name',
       'away_club_name', 'aggregate', 'competition_type'],
      dtype='object')

In [559]:
# Select desired columns
games_df_cleaned = games_df_cleaned.drop(columns=['home_club_position', 'away_club_position', 'stadium', 'attendance',
                                                    'referee', 'url', 'home_club_name', 'away_club_name', 'aggregate', 'competition_type'])

In [560]:
# Filter games that were played in the 2015/2016 season
games_df_cleaned = games_df_cleaned[games_df_cleaned['season'] == 2015]

In [561]:
# Convert date column to datetime format
games_df_cleaned['date'] = pd.to_datetime(games_df_cleaned['date'], format='%Y-%m-%d')

In [562]:
# View basic statistical summary
games_df_cleaned.describe()

,game_id,season,date,home_club_id,away_club_id,home_club_goals,away_club_goals
count,5.708000e+03,5708.0,5708,5708.000000,5708.000000,5708.000000,5708.000000
mean,2.609463e+06,2015.0,2015-12-07 16:29:10.805886464,3821.778206,3623.733707,1.544849,1.265242
min,2.576512e+06,2015.0,2015-06-30 00:00:00,3.000000,2.000000,0.000000,0.000000
25%,2.588908e+06,2015.0,2015-09-22 00:00:00,338.000000,338.000000,0.000000,0.000000
50%,2.604242e+06,2015.0,2015-12-03 00:00:00,982.000000,979.000000,1.000000,1.000000
75%,2.626455e+06,2015.0,2016-02-23 06:00:00,2995.000000,2944.000000,2.000000,2.000000
max,3.282963e+06,2015.0,2016-05-29 00:00:00,68608.000000,68608.000000,15.000000,12.000000
std,2.877316e+04,0.0,NaN,8123.608987,7837.346225,1.413316,1.271627


In [563]:
# View DataFrame structure, data types and missing values
games_df_cleaned.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 5708 entries, 296 to 74015
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   game_id                 5708 non-null   int64         
 1   competition_id          5708 non-null   object        
 2   season                  5708 non-null   int64         
 3   round                   5708 non-null   object        
 4   date                    5708 non-null   datetime64[ns]
 5   home_club_id            5708 non-null   float64       
 6   away_club_id            5708 non-null   float64       
 7   home_club_goals         5708 non-null   float64       
 8   away_club_goals         5708 non-null   float64       
 9   home_club_manager_name  5658 non-null   object        
 10  away_club_manager_name  5658 non-null   object        
 11  home_club_formation     5652 non-null   object        
 12  away_club_formation     5665 non-null   object    

Columns with null values are not necessarily essential for cleaning.

In [564]:
# Convert date column to datetime format
games_df_cleaned['date'] = pd.to_datetime(games_df_cleaned['date'], format='%Y-%m-%d')

In [565]:
# Check duplicate data
games_df_cleaned.duplicated().sum()

np.int64(0)

In [566]:
# Data after cleaning
games_df_cleaned.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_manager_name,away_club_manager_name,home_club_formation,away_club_formation
296,2577883,GB1,2015,2. Matchday,2015-08-15,2288.0,762.0,2.0,0.0,Garry Monk,Steve McClaren,4-3-3 Defending,4-2-3-1
297,2577939,GB1,2015,4. Matchday,2015-08-29,989.0,1003.0,1.0,1.0,Eddie Howe,Claudio Ranieri,4-4-1-1,4-1-4-1
298,2577948,GB1,2015,4. Matchday,2015-08-29,631.0,873.0,1.0,2.0,José Mourinho,Alan Pardew,4-2-3-1,4-4-1-1
299,2577961,GB1,2015,7. Matchday,2015-09-26,31.0,405.0,3.0,2.0,Brendan Rodgers,Tim Sherwood,3-5-2,4-3-2-1
300,2577967,GB1,2015,7. Matchday,2015-09-26,762.0,631.0,2.0,2.0,Steve McClaren,José Mourinho,4-4-2,4-2-3-1


### game_events_df

In [567]:
# Create a copy of the DataFrame for data cleaning
game_events_df_cleaned = game_events_df.copy()

In [568]:
# Quickly preview the table data
game_events_df_cleaned.head()

,game_event_id,date,game_id,minute,type,club_id,player_id,description,player_in_id,player_assist_id
0,2f41da30c471492e7d4a984951671677,2012-08-05,2211607,77,Cards,610,4425,"1. Yellow card , Mass confrontation",NaN,NaN
1,a72f7186d132775f234d3e2f7bc0ed5b,2012-08-05,2211607,77,Cards,383,33210,"1. Yellow card , Mass confrontation",NaN,NaN
2,b2d721eaed4692a5c59a92323689ef18,2012-08-05,2211607,3,Goals,383,36500,", Header, 1. Tournament Goal Assist: , Corner,...",NaN,56416.0
3,aef768899cedac0c9a650980219075a2,2012-08-05,2211607,53,Goals,383,36500,", Right-footed shot, 2. Tournament Goal Assist...",NaN,146258.0
4,5d6d9533023057b6619ecd145a038bbe,2012-08-05,2211607,74,Substitutions,383,36500,", Not reported",49499.0,NaN


In [569]:
# Filter games by games_df_cleaned
game_events_df_cleaned = game_events_df_cleaned[game_events_df_cleaned['game_id'].isin(games_df_cleaned['game_id'])]

In [570]:
# View basic statistical summary
game_events_df_cleaned.describe()

,game_id,minute,club_id,player_id,player_in_id,player_assist_id
count,7.357400e+04,73574.000000,73574.000000,7.357400e+04,32219.000000,11778.000000
mean,2.609596e+06,60.625574,3705.609482,1.137502e+05,136918.563891,106394.128120
std,2.851948e+04,23.632321,8008.980209,8.992317e+04,102140.049962,81364.305082
min,2.576512e+06,-1.000000,2.000000,1.000000e+01,10.000000,10.000000
25%,2.588979e+06,46.000000,338.000000,4.478700e+04,51969.000000,44162.000000
50%,2.604258e+06,66.000000,976.000000,8.531400e+04,112930.000000,79893.000000
75%,2.624875e+06,79.000000,2990.000000,1.675560e+05,204103.000000,157539.000000
max,3.282963e+06,120.000000,68608.000000,1.042103e+06,847815.000000,415859.000000


In [571]:
# Check date with minute = -1
game_events_df_cleaned[game_events_df_cleaned['minute'] < 0]

,game_event_id,date,game_id,minute,type,club_id,player_id,description,player_in_id,player_assist_id
216315,cc070e68ae0e5a0a3c34344f10efb20c,2015-08-01,2576512,-1,Shootout,27,2219,", Scored",NaN,NaN
216317,b7b729769dc348a02d96ea959da2d8ef,2015-08-01,2576512,-1,Shootout,27,4360,", Scored",NaN,NaN
216318,e94d7e85281568cc620b494ea39bcd83,2015-08-01,2576512,-1,Shootout,27,7476,", Saved",NaN,NaN
216321,b752d960eaffd2637b69439d92bcb11a,2015-08-01,2576512,-1,Shootout,82,34557,", Scored",NaN,NaN
216322,54238f2405f83c51e8894144f2c4205d,2015-08-01,2576512,-1,Shootout,82,36182,", Scored",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
289862,793162f3ec6dfe9879e824ed6f69f577,2016-05-28,2690027,-1,Shootout,418,39381,", Scored",NaN,NaN
289863,97f14f810d278c272c8e51632c84fc88,2016-05-28,2690027,-1,Shootout,418,44501,", Scored",NaN,NaN
289866,206e9c4f1a0476c7faa3c765e94d8e9a,2016-05-28,2690027,-1,Shootout,13,125781,", Scored",NaN,NaN
289871,00590d14223e6c5c1a9b1246364ad11f,2016-05-28,2690027,-1,Shootout,13,148928,", Scored",NaN,NaN


That refers to the player who has excecuted the penalty shootout.

In [572]:
# View DataFrame structure, data types and missing values
game_events_df_cleaned.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 73574 entries, 216315 to 572270
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   game_event_id     73574 non-null  object 
 1   date              73574 non-null  object 
 2   game_id           73574 non-null  int64  
 3   minute            73574 non-null  int64  
 4   type              73574 non-null  object 
 5   club_id           73574 non-null  int64  
 6   player_id         73574 non-null  int64  
 7   description       63893 non-null  object 
 8   player_in_id      32219 non-null  float64
 9   player_assist_id  11778 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 6.2+ MB


Columns with null values are not necessarily essential for cleaning.

In [573]:
# Check duplicate data
game_events_df_cleaned.duplicated().sum()

np.int64(0)

In [574]:
# Data after cleaning
game_events_df_cleaned.head()

,game_event_id,date,game_id,minute,type,club_id,player_id,description,player_in_id,player_assist_id
216315,cc070e68ae0e5a0a3c34344f10efb20c,2015-08-01,2576512,-1,Shootout,27,2219,", Scored",NaN,NaN
216316,4358ceb22c14484814f306c957364277,2015-08-01,2576512,49,Goals,27,4360,", Left-footed shot, 1. Tournament Goal Assist:...",NaN,38253.0
216317,b7b729769dc348a02d96ea959da2d8ef,2015-08-01,2576512,-1,Shootout,27,4360,", Scored",NaN,NaN
216318,e94d7e85281568cc620b494ea39bcd83,2015-08-01,2576512,-1,Shootout,27,7476,", Saved",NaN,NaN
216319,bd62e688ee6669f96c60dc7d3143b196,2015-08-01,2576512,71,Cards,82,32213,"1. Yellow card , Foul",NaN,NaN


In [575]:
# Create a function that removes redundant information
import re
def remove_numbered_parts(text):
    if pd.isna(text):
        return text
    parts = [p.strip() for p in text.split(',')]
    filtered = [p for p in parts if not re.match(r'^\d+\.', p.strip())]
    return ', '.join(filtered)

In [576]:
# Apply the function on description
game_events_df_cleaned['description_cleaned'] = game_events_df_cleaned['description'].apply(remove_numbered_parts)

In [577]:
# Extract the information in description_cleaned
game_events_df_cleaned[['foul_types', 'goal_types', 'assist_types']] = game_events_df_cleaned['description_cleaned'].str.split(',', expand=True)

In [578]:
# Classify the information in foul_types that is accidentally in goal_types

mask_1 = game_events_df_cleaned['type'] == 'Cards'

game_events_df_cleaned.loc[mask_1, 'foul_types'] = game_events_df_cleaned.loc[mask_1, 'goal_types']

game_events_df_cleaned.loc[mask_1, 'goal_types'] = np.nan

In [579]:
# Classify the information in assist_types that is accidentally in goal_types

mask_2 = game_events_df_cleaned['goal_types'].isin([' Pass', ' Corner', ' Cross', ' Throw-in', ' Free kick'])

game_events_df_cleaned.loc[mask_2, 'assist_types'] = game_events_df_cleaned.loc[mask_2, 'goal_types']

game_events_df_cleaned.loc[mask_2, 'goal_types'] = np.nan

In [580]:
# Extract combined information in goal_types to assist_types
mask_3 = game_events_df_cleaned['type'] == 'Goals'
has_colon = game_events_df_cleaned['goal_types'].notna() & game_events_df_cleaned['goal_types'].str.contains(':')
mask_final = mask_3 & has_colon

game_events_df_cleaned.loc[mask_final, ['goal_types', 'assist_types']] = game_events_df_cleaned.loc[mask_final, 'goal_types'].str.split(':', n=1, expand=True)

In [581]:
# Remove redundant information from the assist_types column

mask_4 = game_events_df_cleaned['assist_types'] == ' Penalty: Fouled player'

game_events_df_cleaned.loc[mask_4, 'assist_types'] = game_events_df_cleaned.loc[mask_4, 'assist_types'].str.split(':', n=1).str[1]

In [582]:
# Standardize all missing values

game_events_df_cleaned.replace([None, '', ' Not reported'], np.nan, inplace=True)

In [583]:
# Trim string values in 3 new columns

cols = ['foul_types', 'goal_types', 'assist_types']

game_events_df_cleaned[cols] = game_events_df_cleaned[cols].apply(lambda x: x.str.strip())

In [584]:
# Remove not interested columns

game_events_df_cleaned.drop(columns= ['date', 'description', 'description_cleaned'], inplace = True)

### game_lineups_df

In [585]:
# Create a copy of the DataFrame for data cleaning
game_lineups_df_cleaned = game_lineups_df.copy()

In [586]:
# Quickly preview the table data
game_lineups_df_cleaned.head()

,game_lineups_id,date,game_id,player_id,club_id,player_name,type,position,number,team_captain
0,b2dbe01c3656b06c8e23e9de714e26bb,2013-07-27,2317258,1443,610,Christian Poulsen,substitutes,Defensive Midfield,5,0
1,b50a3ec6d52fd1490aab42042ac4f738,2013-07-27,2317258,5017,610,Niklas Moisander,starting_lineup,Centre-Back,4,0
2,7d890e6d0ff8af84b065839966a0ec81,2013-07-27,2317258,9602,1090,Maarten Martens,substitutes,Left Winger,11,0
3,8c355268678b9bbc7084221b1f0fde36,2013-07-27,2317258,12282,610,Daley Blind,starting_lineup,Left-Back,17,0
4,76193074d549e5fdce4cdcbba0d66247,2013-07-27,2317258,25427,1090,Roy Beerens,starting_lineup,Right Winger,23,0


In [587]:
# Filter games by games_df_cleaned
game_lineups_df_cleaned = game_lineups_df_cleaned[game_lineups_df_cleaned['game_id'].isin(games_df_cleaned['game_id'])]

In [588]:
# View basic statistical summary
game_lineups_df_cleaned.describe()

,game_id,player_id,club_id,team_captain
count,2.075590e+05,2.075590e+05,207559.000000,207559.000000
mean,2.609540e+06,1.201819e+05,3617.881128,0.048642
std,2.869182e+04,9.631748e+04,7814.754537,0.215118
min,2.576512e+06,1.000000e+01,2.000000,0.000000
25%,2.588935e+06,4.456700e+04,338.000000,0.000000
50%,2.604232e+06,9.047800e+04,976.000000,0.000000
75%,2.624878e+06,1.813390e+05,2944.000000,0.000000
max,3.282963e+06,1.203976e+06,68608.000000,1.000000


In [589]:
# View DataFrame structure, data types and missing values
game_lineups_df_cleaned.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 207559 entries, 417127 to 1425682
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   game_lineups_id  207559 non-null  object
 1   date             207559 non-null  object
 2   game_id          207559 non-null  int64 
 3   player_id        207559 non-null  int64 
 4   club_id          207559 non-null  int64 
 5   player_name      207559 non-null  object
 6   type             207559 non-null  object
 7   position         207557 non-null  object
 8   number           207559 non-null  object
 9   team_captain     207559 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 17.4+ MB


In [590]:
# Recheck data type of number

game_lineups_df_cleaned['number'].unique()

array(['21', '10', '14', '4', '1', '17', '25', '20', '13', '3', '8', '11',
       '23', '9', '7', '15', '26', '5', '12', '27', '6', '19', '34', '31',
       '28', '18', '32', '22', '33', '39', '36', '2', '24', '16', '30',
       '35', '29', '38', '37', '40', '41', '59', '46', '44', '42', '45',
       '50', '72', '76', '43', '70', '62', '58', '49', '53', '48', '56',
       '54', '77', '47', '52', '69', '75', '51', '61', '68', '64', '74',
       '63', '-', '99', '88', '55', '90', '80', '98', '79', '91', '57',
       '87', '94', '71', '66', '97', '93', '84', '83', '89', '92', '78',
       '96', '81', '86', '95', '73', '60', '65', '85', '67', '82'],
      dtype=object)

There is '-' in the numeric values, but since we will not use this column for mathematical calculations, we're going to keep the data type as is.

In [591]:
# Locate missing data

game_lineups_df_cleaned[game_lineups_df_cleaned['position'].isnull()]

,game_lineups_id,date,game_id,player_id,club_id,player_name,type,position,number,team_captain
501768,8e0d3d1774f12f52c269a21edc837287,2015-08-26,2597514,415060,6601,Jimmy Theelen,starting_lineup,NaN,-,0
501771,22014173b77618f434041e511467c704,2015-08-26,2597514,415063,6601,Bart Zeevenhoven,substitutes,NaN,-,0


Keep it for later consideration

In [592]:
# Check duplicate data
game_lineups_df_cleaned.duplicated().sum()

np.int64(0)

In [593]:
# Drop redundant columns
game_lineups_df_cleaned = game_lineups_df_cleaned.drop(columns = ['date', 'player_name'])

### appearances_df

In [594]:
# Create a copy of the DataFrame for data cleaning
appearances_df_cleaned = appearances_df.copy()

In [595]:
# Quickly preview the table data
appearances_df_cleaned.head()

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45
3,2234418_73333,2234418,73333,1274,6646,2012-07-05,Vegar Hedenstad,ELQ,0,0,0,0,90
4,2234421_122011,2234421,122011,195,3008,2012-07-05,Markus Henriksen,ELQ,0,0,0,1,90


In [596]:
# Filter games by games_df_cleaned
appearances_df_cleaned = appearances_df_cleaned[appearances_df_cleaned['game_id'].isin(games_df_cleaned['game_id'])]

In [597]:
# View basic statistical summary
appearances_df_cleaned.describe()

,game_id,player_id,player_club_id,player_current_club_id,yellow_cards,red_cards,goals,assists,minutes_played
count,1.314360e+05,131436.000000,131436.000000,131436.000000,131436.000000,131436.000000,131436.000000,131436.000000,131436.000000
mean,2.607947e+06,105581.462301,2681.397745,3940.585433,0.157339,0.003842,0.097568,0.077346,71.530365
std,2.628653e+04,83546.778237,6627.141848,10296.325136,0.377378,0.061866,0.335414,0.289396,28.794895
min,2.576512e+06,10.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.587029e+06,41416.000000,281.000000,368.000000,0.000000,0.000000,0.000000,0.000000,60.000000
50%,2.604061e+06,78507.000000,825.000000,984.000000,0.000000,0.000000,0.000000,0.000000,90.000000
75%,2.618791e+06,157543.000000,2448.000000,2759.000000,0.000000,0.000000,0.000000,0.000000,90.000000
max,2.693536e+06,435666.000000,68608.000000,110302.000000,2.000000,1.000000,5.000000,4.000000,120.000000


In [598]:
# View DataFrame structure, data types and missing values
appearances_df_cleaned.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 131436 entries, 389923 to 521358
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   appearance_id           131436 non-null  object
 1   game_id                 131436 non-null  int64 
 2   player_id               131436 non-null  int64 
 3   player_club_id          131436 non-null  int64 
 4   player_current_club_id  131436 non-null  int64 
 5   date                    131436 non-null  object
 6   player_name             131436 non-null  object
 7   competition_id          131436 non-null  object
 8   yellow_cards            131436 non-null  int64 
 9   red_cards               131436 non-null  int64 
 10  goals                   131436 non-null  int64 
 11  assists                 131436 non-null  int64 
 12  minutes_played          131436 non-null  int64 
dtypes: int64(9), object(4)
memory usage: 14.0+ MB


In [599]:
# Check duplicate data
appearances_df_cleaned.duplicated().sum()

np.int64(0)

In [600]:
# Remove not interested columns
appearances_df_cleaned.drop(columns = ['player_current_club_id', 'date', 'player_name', 'competition_id'], inplace = True, errors = False)

### players_df

In [601]:
# Create a copy of the DataFrame for data cleaning
players_df_cleaned = players_df.copy()

In [602]:
# Quickly preview the table data
players_df_cleaned.head()

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,...,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0


In [603]:
# Filter players by game_events_df_cleaned
players_df_cleaned = players_df_cleaned[players_df_cleaned['player_id'].isin(game_events_df_cleaned['player_id'])]

In [604]:
# Filter players who are still available next season
players_df_cleaned = players_df_cleaned[~(players_df_cleaned['last_season'] <= 2015)]

In [605]:
# Keep only players whose sub_position indicates a forward position
players_df_cleaned = players_df_cleaned[players_df_cleaned['sub_position'].isin(['Left Winger', 'Right Winger', 'Centre-Forward', 'Second Striker'])]

In [606]:
# Remove redundant columns
players_df_cleaned = players_df_cleaned.drop(columns = ['first_name', 'last_name', 'last_season', 'current_club_id', 'player_code', 'position' ,'url', 'current_club_domestic_competition_id',
                                                        'current_club_name', 'market_value_in_eur',
                                                        'highest_market_value_in_eur', 'contract_expiration_date'])

In [607]:
# Double-check the 2 players whose cases were reconsidered
players_df_cleaned[players_df_cleaned['player_id'].isin([415060, 415063])]

,player_id,name,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,foot,height_in_cm,agent_name,image_url


They are not forwards, so they were removed

In [608]:
# Convert date_of_birth column to datetime format
players_df_cleaned['date_of_birth'] = players_df_cleaned['date_of_birth'].str.split(' ').str[0]
players_df_cleaned['date_of_birth'] = pd.to_datetime(players_df_cleaned['date_of_birth'], format='%Y-%m-%d')

In [609]:
# View basic statistical summary
players_df_cleaned.describe()

,player_id,date_of_birth,height_in_cm
count,1747.000000,1745,1732.000000
mean,141793.411563,1991-02-14 07:38:49.169054464,181.065820
min,532.000000,1973-08-27 00:00:00,163.000000
25%,59000.500000,1988-08-07 00:00:00,177.000000
50%,123397.000000,1991-05-05 00:00:00,181.000000
75%,207971.500000,1994-01-01 00:00:00,186.000000
max,425707.000000,2000-04-13 00:00:00,203.000000
std,96952.991905,NaN,6.555478


In [610]:
# View DataFrame structure, data types and missing values
players_df_cleaned.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 1747 entries, 21 to 21865
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   player_id               1747 non-null   int64         
 1   name                    1747 non-null   object        
 2   country_of_birth        1699 non-null   object        
 3   city_of_birth           1729 non-null   object        
 4   country_of_citizenship  1717 non-null   object        
 5   date_of_birth           1745 non-null   datetime64[ns]
 6   sub_position            1747 non-null   object        
 7   foot                    1722 non-null   object        
 8   height_in_cm            1732 non-null   float64       
 9   agent_name              980 non-null    object        
 10  image_url               1747 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 163.8+ KB


It seems that all missing values are due to the inability to collect information.

In [611]:
# Check duplicate data
players_df_cleaned.duplicated().sum()

np.int64(0)

### player_valuations_df

In [612]:
# Create a copy of the DataFrame for data cleaning
player_valuations_df_cleaned = player_valuations_df.copy()

In [613]:
# Quickly preview the table data
player_valuations_df_cleaned.head()

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,405973,2000-01-20,150000,3057,BE1
1,342216,2001-07-20,100000,1241,SC1
2,3132,2003-12-09,400000,126,TR1
3,6893,2003-12-15,900000,984,GB1
4,10,2004-10-04,7000000,398,IT1


In [614]:
# Filter player_id by players_df_cleaned
player_valuations_df_cleaned = player_valuations_df_cleaned[player_valuations_df_cleaned['player_id'].isin(players_df_cleaned['player_id'])]

In [615]:
# Remove redundant columns
player_valuations_df_cleaned = player_valuations_df_cleaned[['player_id', 'date', 'market_value_in_eur']]

In [616]:
# View basic statistical summary
player_valuations_df_cleaned.describe()

,player_id,market_value_in_eur
count,46124.000000,4.612400e+04
mean,127334.156123,4.756146e+06
std,90817.197350,1.164481e+07
min,532.000000,1.000000e+04
25%,51587.000000,4.000000e+05
50%,99863.000000,1.000000e+06
75%,188131.000000,4.000000e+06
max,425707.000000,2.000000e+08


In [617]:
# Convert date column to datetime format
player_valuations_df_cleaned['date'] = pd.to_datetime(player_valuations_df_cleaned['date'], format='%Y-%m-%d')

In [618]:
# View DataFrame structure, data types and missing values
player_valuations_df_cleaned.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 46124 entries, 24 to 496489
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   player_id            46124 non-null  int64         
 1   date                 46124 non-null  datetime64[ns]
 2   market_value_in_eur  46124 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.4 MB


In [619]:
# Check duplicate data
player_valuations_df_cleaned.duplicated().sum()

np.int64(0)

### transfers_df

In [620]:
# Create a copy of the DataFrame for data cleaning
transfers_df_cleaned = transfers_df.copy()

In [621]:
# Quickly preview the table data
transfers_df_cleaned.head()

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name
0,16136,2026-07-01,26/27,417,123,OGC Nice,Retired,NaN,500000.0,Dante
1,1138758,2026-07-01,26/27,336,631,Sporting CP,Chelsea,52140000.0,45000000.0,Geovany Quenda
2,195778,2026-06-30,25/26,79,27,VfB Stuttgart,Bayern Munich,0.0,12000000.0,Alexander Nübel
3,569033,2026-06-30,25/26,39,27,1.FSV Mainz 05,Bayern Munich,0.0,4000000.0,Armindo Sieb
4,626913,2026-06-30,25/26,398,380,Lazio,Salernitana,0.0,15000000.0,Boulaye Dia


In [622]:
# Convert transfer_date column to datetime format
transfers_df_cleaned['transfer_date'] = pd.to_datetime(transfers_df_cleaned['transfer_date'], format='%Y-%m-%d')

In [623]:
# Filter transfers that occurred before the end of the 2015/2016 season
transfers_df_cleaned = transfers_df_cleaned[transfers_df_cleaned['transfer_date'] < np.max(games_df_cleaned['date'])]

In [624]:
# Filter players by players_df_cleaned
transfers_df_cleaned = transfers_df_cleaned[transfers_df_cleaned['player_id'].isin(players_df_cleaned['player_id'])]

In [625]:
# Remove redundant column
transfers_df_cleaned.drop(columns=['player_name'])

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur
59722,246963,2016-05-16,15/16,2999,33614,Hamilton Acad.,Norwich U21,0.0,100000.0
59736,203869,2016-05-01,15/16,3026,33616,Morton,Swansea U21,0.0,75000.0
59748,237623,2016-04-29,15/16,32036,515,Avangard K.,Without Club,NaN,100000.0
59784,252689,2016-03-31,15/16,2410,272,CSKA Moscow,AIK,0.0,800000.0
59794,59290,2016-03-23,15/16,12302,29109,Naftovyk,Girnyk-Sport,0.0,125000.0
...,...,...,...,...,...,...,...,...,...
79579,45660,2000-07-01,00/01,53081,38759,Ottensen Yth.,Altona 93 Yth.,0.0,NaN
79585,58358,2000-07-01,00/01,42290,18936,TSV Pähl Yth.,B. München Yth.,NaN,NaN
79604,33706,1999-07-01,99/00,72277,61166,Real Formação,Real SC U155,NaN,NaN
79608,55619,1999-07-01,99/00,64983,32791,Zuidvogels Yth.,Ajax Youth,0.0,NaN


In [626]:
# View basic statistical summary
transfers_df_cleaned.describe()

,player_id,transfer_date,from_club_id,to_club_id,transfer_fee,market_value_in_eur
count,2626.000000,2626,2626.000000,2626.000000,1.670000e+03,1.332000e+03
mean,156819.269612,2012-02-18 16:22:39.939070720,15063.178979,9543.050647,1.511022e+06,2.859948e+06
min,18644.000000,1998-07-01 00:00:00,2.000000,2.000000,0.000000e+00,1.300000e+04
25%,88683.000000,2010-07-01 00:00:00,985.000000,515.000000,0.000000e+00,2.000000e+05
50%,146370.000000,2012-07-19 12:00:00,6767.000000,2517.000000,0.000000e+00,8.000000e+05
75%,215122.000000,2014-07-01 00:00:00,19730.000000,11457.500000,0.000000e+00,2.500000e+06
max,412513.000000,2016-05-16 00:00:00,125130.000000,121251.000000,7.500000e+07,6.000000e+07
std,83440.762045,NaN,20880.083001,15745.795581,5.592455e+06,5.749518e+06


In [627]:
# View DataFrame structure, data types and missing values
transfers_df_cleaned.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 2626 entries, 59722 to 79624
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   player_id            2626 non-null   int64         
 1   transfer_date        2626 non-null   datetime64[ns]
 2   transfer_season      2626 non-null   object        
 3   from_club_id         2626 non-null   int64         
 4   to_club_id           2626 non-null   int64         
 5   from_club_name       2626 non-null   object        
 6   to_club_name         2626 non-null   object        
 7   transfer_fee         1670 non-null   float64       
 8   market_value_in_eur  1332 non-null   float64       
 9   player_name          2626 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 225.7+ KB


In [628]:
# Locate missing value rows
transfers_df_cleaned[transfers_df_cleaned['transfer_fee'].isna()].head(10)

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name
59748,237623,2016-04-29,15/16,32036,515,Avangard K.,Without Club,NaN,100000.0,Oleksandr Filippov
59856,144304,2016-02-26,15/16,448,4768,Vojvodina,CC Yatai,NaN,1000000.0,Ognjen Ozegovic
60342,315779,2016-01-04,15/16,1464,16,B. Dortmund U19,Bor. Dortmund,NaN,NaN,Christian Pulisic
60343,330876,2016-01-04,15/16,15021,20,Bursaspor U21,Bursaspor,NaN,NaN,Kubilay Kanatsızkuş
60416,258923,2016-01-01,15/16,5242,985,Man Utd U18,Man Utd,NaN,NaN,Marcus Rashford
60422,279242,2016-01-01,15/16,6189,40280,Spartak II,Spartak 2,NaN,75000.0,Zelimkhan Bakaev
60453,342229,2016-01-01,15/16,9669,162,Monaco U19,Monaco,NaN,50000.0,Kylian Mbappé
60455,343492,2016-01-01,15/16,14280,12414,Dnipro U19,Dnipro II,NaN,NaN,Oleg Kozhushko
60463,349107,2016-01-01,15/16,12517,6414,Metalist II,Metalist Kharkiv,NaN,NaN,Oleg Synytsya
60464,351478,2016-01-01,15/16,9899,681,Real Sociedad B,Real Sociedad,NaN,NaN,Mikel Oyarzabal


Most of the missing values are due to transfers that occurred when players’ contracts expired, promotions from youth academies, or cases where players were not signed by any club after their contracts ended.

In [629]:
# Check duplicate data
transfers_df_cleaned.duplicated().sum()

np.int64(0)

### clubs_df

In [630]:
# Create a copy of the DataFrame for data cleaning
clubs_df_cleaned = clubs_df.copy()

In [631]:
# Quickly preview the table data
clubs_df_cleaned.head()

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,NaN,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,NaN,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,30,26.6,15,50.0,8,Beşiktaş Park,42445,€-25.26m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,26.3,18,69.2,17,Olimpico di Roma,70634,€-76.90m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,30,25.5,21,70.0,18,Tottenham Hotspur Stadium,62850,€-120.05m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...


In [632]:
# Filter clubs by transfers_df
clubs_df_cleaned = clubs_df_cleaned[(clubs_df_cleaned['club_id'].isin(transfers_df['from_club_id']))
                                    |(clubs_df_cleaned['club_id'].isin(transfers_df['to_club_id']))]

In [633]:
# Remove redundant columns
clubs_df_cleaned = clubs_df_cleaned[['club_id', 'club_code', 'name', 'domestic_competition_id']]

In [634]:
# View basic statistical summary
clubs_df_cleaned.describe()

,club_id
count,438.000000
mean,6294.600457
std,14603.578260
min,3.000000
25%,432.750000
50%,1153.000000
75%,3583.500000
max,110302.000000


In [635]:
# View DataFrame structure, data types and missing values
clubs_df_cleaned.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 438 entries, 0 to 438
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   club_id                  438 non-null    int64 
 1   club_code                438 non-null    object
 2   name                     438 non-null    object
 3   domestic_competition_id  438 non-null    object
dtypes: int64(1), object(3)
memory usage: 17.1+ KB


In [636]:
# Check duplicated value
clubs_df_cleaned.duplicated().sum()

np.int64(0)

### competitions_df

In [637]:
# Create a copy of the DataFrame for data cleaning
competitions_df_cleaned = competitions_df.copy()

In [638]:
# Quickly preview the table data
competitions_df_cleaned.head()

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url,is_major_national_league
0,CIT,italy-cup,italy-cup,domestic_cup,domestic_cup,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/italy-cup/star...,False
1,NLSC,johan-cruijff-schaal,johan-cruijff-schaal,domestic_super_cup,other,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/johan-cruijff-...,False
2,GRP,kypello-elladas,kypello-elladas,domestic_cup,domestic_cup,56,Greece,GR1,europa,https://www.transfermarkt.co.uk/kypello-ellada...,False
3,POSU,supertaca-candido-de-oliveira,supertaca-candido-de-oliveira,domestic_super_cup,other,136,Portugal,PO1,europa,https://www.transfermarkt.co.uk/supertaca-cand...,False
4,RUSS,russian-super-cup,russian-super-cup,domestic_super_cup,other,141,Russia,RU1,europa,https://www.transfermarkt.co.uk/russian-super-...,False


In [639]:
# View basic statistical summary
competitions_df_cleaned.describe()

,country_id
count,44.000000
mean,94.863636
std,70.518871
min,-1.000000
25%,39.000000
50%,98.500000
75%,157.000000
max,190.000000


In [640]:
# View DataFrame structure, data types and missing values
competitions_df_cleaned.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   competition_id            44 non-null     object
 1   competition_code          44 non-null     object
 2   name                      44 non-null     object
 3   sub_type                  44 non-null     object
 4   type                      44 non-null     object
 5   country_id                44 non-null     int64 
 6   country_name              36 non-null     object
 7   domestic_league_code      36 non-null     object
 8   confederation             44 non-null     object
 9   url                       44 non-null     object
 10  is_major_national_league  44 non-null     bool  
dtypes: bool(1), int64(1), object(9)
memory usage: 3.6+ KB


In [641]:
# Locate missing value rows
competitions_df_cleaned[competitions_df_cleaned['country_name'].isna()]

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url,is_major_national_league
6,USC,uefa-super-cup,uefa-super-cup,uefa_super_cup,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-super-cup...,False
8,EL,europa-league,europa-league,europa_league,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/europa-league/...,False
17,UCOL,uefa-conference-league,uefa-conference-league,uefa_europa_conference_league,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-conferenc...,False
18,ELQ,europa-league-qualifikation,europa-league-qualifikation,europa_league_qualifying,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/europa-league-...,False
21,ECLQ,uefa-europa-conference-league-qualifikation,uefa-europa-conference-league-qualifikation,uefa_europa_conference_league_qualifiers,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-europa-co...,False
27,CL,uefa-champions-league,uefa-champions-league,uefa_champions_league,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-champions...,False
32,KLUB,fifa-klub-wm,fifa-klub-wm,fifa_club_world_cup,other,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/fifa-klub-wm/s...,False
38,CLQ,uefa-champions-league-qualifikation,uefa-champions-league-qualifikation,uefa_champions_league_qualifying,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-champions...,False


All missing values are due to that tournament being international.

In [642]:
# Check duplicate data
competitions_df_cleaned.duplicated().sum()

np.int64(0)

## Load cleaned data into MS SQL Server

In [643]:
# Connection information
server = "COMPUTER"
database = "FootballDB"

# Connect to SQL Server using the pymssql driver
conn_str = (
    f"mssql+pyodbc://@{server}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)
engine = create_engine(conn_str)

# Push each table's data into SQL Server
games_df_cleaned.to_sql("games", con=engine, index=False, if_exists="replace")
game_events_df_cleaned.to_sql("game_events", con=engine, index=False, if_exists="replace")
game_lineups_df_cleaned.to_sql("game_lineups", con=engine, index=False, if_exists="replace")
appearances_df_cleaned.to_sql("appearances", con=engine, index=False, if_exists="replace")
players_df_cleaned.to_sql("players", con=engine, index=False, if_exists="replace")
player_valuations_df_cleaned.to_sql("player_valuations", con=engine, index=False, if_exists="replace")
transfers_df_cleaned.to_sql("transfers", con=engine, index=False, if_exists="replace")
clubs_df_cleaned.to_sql("clubs", con=engine, index=False, if_exists="replace")
competitions_df_cleaned.to_sql("competitions", con=engine, index=False, if_exists="replace")

44